In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output

Data Visualization

In [ ]:
#Visualize each run in a specific FCD file
import geopandas as gpd
import pandas as pd
import plotly.express as px
from pyproj import Transformer
from shapely import wkt
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Load the GeoDataFrame
gdf = gpd.read_parquet('new_cleaned/november_traffic.parquet')

# Check and convert geometry column if necessary
if isinstance(gdf['geometry'].iloc[0], str):
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)

# Define a transformer from UTM Zone 32N to WGS 84 (latitude and longitude)
transformer = Transformer.from_crs("epsg:32632", "epsg:4326", always_xy=True)

# Apply transformation to convert coordinates
gdf['longitude'], gdf['latitude'] = transformer.transform(gdf['geometry'].x, gdf['geometry'].y)

# Reset index to ensure clean, sequential indices before processing
gdf_reset = gdf.reset_index()

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Bus Run Location Viewer"),
    dcc.Dropdown(
        id='run-dropdown',
        options=[{'label': str(i), 'value': i} for i in gdf_reset['run'].unique()],
        value=gdf_reset['run'].unique()[0],
    ),
    dcc.Graph(id='map-plot')
])

# Define callback to update graph based on selected run
@app.callback(
    Output('map-plot', 'figure'),
    [Input('run-dropdown', 'value')]
)
def update_map(selected_run):
    filtered_data = gdf_reset[gdf_reset['run'] == selected_run]
    if filtered_data.empty:
        raise dash.exceptions.PreventUpdate  # Prevents update if no data found

    fig = px.scatter_mapbox(
        filtered_data,
        lat='latitude',
        lon='longitude',
        color='run',
        hover_name='run',
        hover_data=['speed', 'utcTime'],
        title=f'Bus Locations for Run {selected_run}',
        mapbox_style='open-street-map',
        zoom=13  # Set a default zoom level
    )
    fig.update_layout(mapbox_style="open-street-map", margin={"r":0,"t":0,"l":0,"b":0})
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
